In [1]:
import dask.array as da
import xarray as xr
import numpy as np
import pandas as pd
import dask.array as da
from typing import Union
from dask.diagnostics import ProgressBar
from dask.distributed import Client
client = Client() 

In [2]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 7
Total threads: 14,Total memory: 23.06 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46131,Workers: 7
Dashboard: http://127.0.0.1:8787/status,Total threads: 14
Started: Just now,Total memory: 23.06 GiB
Comm: tcp://127.0.0.1:46057,Total threads: 2
Dashboard: http://127.0.0.1:45681/status,Memory: 3.29 GiB
Nanny: tcp://127.0.0.1:34927,


In [3]:
ZARR_PATH = "era5_derived.zarr"
CLIM_CLASS = 2 # LA County NFDRS Climate Class

In [4]:
ds = xr.open_dataset(ZARR_PATH, chunks="auto", consolidated=True)
ds

/home/mitch/mamba/envs/pygeo/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


<xarray.Dataset> Size: 16GB
Dimensions:     (latitude: 26, valid_time: 648988, longitude: 30)
Coordinates:
  * latitude    (latitude) float64 208B 38.78 38.53 38.28 ... 33.03 32.78 32.53
  * longitude   (longitude) float64 240B -124.4 -124.2 -123.9 ... -117.4 -117.2
  * valid_time  (valid_time) datetime64[ns] 5MB 1951-01-01 ... 2025-01-13T03:...
Data variables:
    tp          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    fosberg     (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    u10         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    t2m         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    v10         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    wind        (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    rh          (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
    d2m         (valid_time, latitude, longitude) float32 2GB dask.array<chunksize=(43000, 26, 30), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-01-16T21:06 GRIB to CDM+CF via cfgrib-0.9.1...

In [5]:
def get_daylight(time, time_dims, lat, lat_dims):
    """
    Calculate the number of daylight hours for a given day of the year and latitude.

    Args:
        time (Union[da.Array, np.ndarray]): datetime
        lat: (Union[da.Array, np.ndarray]), latitude in decimal degrees

    Returns:
        daylit: dask.array or numpy array, the number of daylight hours
    """
   # Ensure the day of the year is within bounds
    j_date = time.dt.dayofyear
    j_date = da.clip(j_date, 1, 365)
    # Convert latitude to radians
    phi = da.radians(lat.values)
    # Calculate solar declination angle (in radians)
    decl = 0.41008 * da.sin((j_date - 82) * da.radians(1))
    # Reshape to enable broadcasting
    phi = lat.expand_dims({"valid_time": time_dims})  # Expand for valid_time
    decl = time.dt.dayofyear.expand_dims({"latitude": lat_dims})  # Expand for latitude
    # Compute daylight hours for all (latitude, declination) combinations
    daylit = 24 * (1 - da.arccos(da.clip(da.tan(da.radians(phi)) * da.tan(da.radians(decl)), -1, 1)) / da.pi)
    # Ensure the result is real (no complex values)
    daylit = da.real(daylit)
    return daylit

def get_eqilibrium_moisture_content(rh, t):
    emc = da.where(
        condition=rh > 50,
        x=21.0606 + 0.005565 * rh**2 - 0.00035 * rh * t - 0.483199 * rh,
        y=rh,
    )
    emc = da.where(
        condition=(rh > 10) & (rh <= 50), 
        x=2.22749 + 0.160107 * rh - 0.014784 * t, 
        y=emc,
    )
    return da.where(
        condition=rh <= 10, 
        x=0.03229 + 0.281073 * rh - 0.000578 * rh * t, 
        y=emc,
    )

def get_daily_aggregation(ds):
    daily_ds = ds.resample(valid_time="1D").map(
        lambda x: xr.Dataset(
            {
                "t2m_max": x["t2m"].max(dim="valid_time"),
                "t2m_min": x["t2m"].min(dim="valid_time"),
                "rh_max": x["rh"].max(dim="valid_time"),
                "rh_min": x["rh"].min(dim="valid_time"),
            }
        )
    )
    emc_daytime = get_eqilibrium_moisture_content(daily_ds["rh_min"], daily_ds["t2m_max"])
    emc_nighttime = get_eqilibrium_moisture_content(daily_ds["rh_max"], daily_ds["t2m_min"])
    daylight = (
        get_daylight(
            daily_ds["valid_time"], 
            daily_ds.dims["valid_time"], 
            daily_ds["latitude"], 
            daily_ds.dims["latitude"],
        )
    )
    daylight = daylight.expand_dims(dim={"longitude": daily_ds.dims["longitude"]}, axis=-1)
    daily_ds["emc"] = (daylight * emc_daytime + (24.0 - daylight) * emc_nighttime) / 24.0

    tp_inches = ds["tp"] * 39.3701
    WETRAT = 0.25 if CLIM_CLASS < 3 else 0.05
    daily_ds["tp_duration"] = (
        ((tp_inches / WETRAT) + 0.49).round()
        .resample(valid_time="1D")
        .sum()
        .clip(max=8)
    )
    
    return daily_ds

In [5]:
def get_100_hr_fuel_moisture(ds):
    # Extract variables
    tp_duration = ds["tp_duration"]  # (valid_time, latitude, longitude)
    emc = ds["emc"]  # (valid_time, latitude, longitude)
    
    # Compute boundary condition
    boundary = ((24 - tp_duration) * emc + (0.5 * tp_duration + 41) * tp_duration) / 24


    fr100 = 0.3156  # Moisture retention factor
    ymc_init = 10  # Initial FM100 (starting moisture content)
    # Define a function for cumulative computation across time
    def fm100_recursive(boundary):
        ymc = da.full_like(boundary[0], ymc_init)  # Initialize with shape (lat, lon)
        fm100 = da.zeros_like(boundary)
    
        for i in range(boundary.shape[0]):  # Iterate over "valid_time"
            fm100[i] = (boundary[i] - ymc) * fr100 + ymc
            ymc = fm100[i]  # Update ymc for next time step
    
        return fm100
    
    # Apply function along "valid_time"
    fm100 = xr.apply_ufunc(
        fm100_recursive, 
        # boundary.chunk({"valid_time": -1}),  
        boundary,
        input_core_dims=[["valid_time"]],  
        output_core_dims=[["valid_time"]],  
        vectorize=True,
        dask="parallelized",
        # dask_gufunc_kwargs={"allow_rechunk": True},
    )
    
    return fm100


In [6]:
def get_1000_hr_fuel_moisture(ds):
    fr1 = 0.3068  # Moisture retention factor

    # Extract variables
    tp_duration = ds["tp_duration"]  # (valid_time, latitude, longitude)
    emc = ds["emc"]  # (valid_time, latitude, longitude)

    # Compute boundary condition (bndry)
    bndry = ((24 - tp_duration) * emc + (2.7 * tp_duration + 76) * tp_duration) / 24

    # Compute 7-day running average boundary condition (bvave)
    bvave = bndry.rolling(valid_time=7, center=False).mean()
    bvave = xr.where(bvave.isnull(), 10 + 5 * CLIM_CLASS, bvave)

    # Define recursive function for fm1000 computation
    def fm1000_recursive(bvave):
        tmois = da.full_like(bvave[:7], 10)  # Shape: (7, lat, lon)
        fm1000 = da.zeros_like(bvave)

        # Iterate over time dimension
        for i in range(bvave.shape[0]):
            # Calculate today's FM1000 using the first element of tmois (most recent stored value)
            fm1000[i] = tmois[0] + (bvave[i] - tmois[0]) * fr1
    
            # Shift tmois values forward (remove first, append latest fm1000 value)
            tmois = da.concatenate([tmois[1:], fm1000[i:i+1]], axis=0)
        return fm1000

    # Apply function along "valid_time"
    fm1000 = xr.apply_ufunc(
        fm1000_recursive, 
        # bvave.chunk({"valid_time": -1}),
        bvave,
        input_core_dims=[["valid_time"]],  
        output_core_dims=[["valid_time"]],  
        vectorize=True,
        dask="parallelized"
    )
    return fm1000

In [ ]:
dfmc_ds = xr.Dataset()

dfmc_ds["fm100"] = get_100_hr_fuel_moisture(daily_ds)
dfmc_ds["fm1000"] = get_1000_hr_fuel_moisture(daily_ds)
dfmc_ds.to_zarr("dfmc.zarr", mode="w")

In [16]:
dfmc_ds

<xarray.Dataset> Size: 338MB
Dimensions:     (valid_time: 27042, latitude: 26, longitude: 30)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 216kB 1951-01-01 ... 2025-01-13
  * latitude    (latitude) float64 208B 32.53 32.78 33.03 ... 38.28 38.53 38.78
  * longitude   (longitude) float64 240B -124.4 -124.2 -123.9 ... -117.4 -117.2
Data variables:
    fm100       (valid_time, latitude, longitude) float64 169MB 10.29 ... 7.439
    fm1000      (valid_time, latitude, longitude) float64 169MB 13.07 ... 10.28